In [1]:
import myfunctions as f
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors as NN

In [2]:
# Functions for point 1

def swiss_roll(n):
    """
    Defines the swiss roll dataset
    """
    data = np.zeros((n,3))
    phi = np.random.uniform(low=1.5*np.pi, high=4.5*np.pi, size=n)
    psi = np.random.uniform(0,10,n)  
    data[:,0]=phi*np.cos(phi)
    data[:,1]=phi*np.sin(phi)
    data[:,2]=psi
    return data

def nearest_neighbors(data, k):
    """
    Returns the k nearest neighbours of each data point.
    Output size: (n,k)
    """
    distance = cdist(data, data, metric='euclidean')
    idx_neighbors = np.array([np.argsort(distance[i,:])[1:k+1]for i in range(np.shape(data)[0])]).astype(int)
    return idx_neighbors

In [3]:
def two_NN(data):
    """
    Given a dataset, returns the intrinsic dimension using 2-nn method.
    """
    neigh = NN(n_neighbors=2).fit(data)
    distances, _ = neigh.kneighbors(return_distance=True)
    distances = distances[(distances > 1.e-5)[:,0],:]
    mu = distances[:,1]/distances[:,0]
    return data.shape[0]/np.sum(np.log(mu))


def pca(data, n_components):
    """
    Performs PCA on a data set
    Input:
    - centered data
    - dimension of projection space
    Output: projected data as a np_array
    """
    if (n_components > np.shape(data)[0]):
        raise Exception('Projecting in a space of higher dimension. Decrease n_components.')
    
    data = data - np.mean(data, axis=0)
    
    cov = np.dot(data.T, data)
    eigenvalues, U = np.linalg.eig(cov)
    idx = np.argsort(eigenvalues)[::-1]
    new_data = np.dot(data, U[:,idx[0:n_components]])
    return new_data

In [4]:
# Functions for point 2

def compute_delta(data, idx_neighbors):
    """
    Computes delta matrix
    """
    n = np.shape(data)[0]
    k = np.shape(idx_neighbors)[1]
    delta = np.array([[np.linalg.norm(data[i,:] - data[idx_neighbors][i,j], 2) for j in range(k)] for i in range(n)])
    return delta

def compute_theta(data, idx_neighbors):
    """
    Computes theta matrix
    """
    n = np.shape(data)[0]
    k = np.shape(idx_neighbors)[1]
    theta = np.zeros((n,k))
    for i in range(n):
        for j in range(k):
            v1 = data[i,:] - data[idx_neighbors][i,j]
            idx = idx_neighbors[i,j]
            neighs = data[idx_neighbors[idx]]
            for l in range(k):
                if idx_neighbors[idx][l] != i:  #controllo per evitare di prendere se stessi come elemento collineare
                    v2 = data[idx_neighbors][i,j] - neighs[l]
                    angle = np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))
                    if angle > theta[i,j]:
                        theta[i,j] = angle
    return theta

In [10]:
class ManifoldSculpting():
    
    def __init__(self, data, n_neigh: int = 5, d: int = None):
        self.data = data
        self.n_neigh = n_neigh
        self.set_d(d)
        
        self.idx_neighbors = nearest_neighbors(self.data, self.n_neigh)
        self.delta = compute_delta(self.data, self.idx_neighbors)
        self.theta = compute_theta(self.data, self.idx_neighbors)
        self.mean_delta = np.mean(self.delta)
        
    def set_d(self, d):
        if d is None:
            self.d = round(two_NN(self.data))
        else:
            self.d = d
            
    def fit():
        # TODO
        # applica pca
        pass

## Point 1

In [17]:
# Number of datapoints
n = 1000

# Generate swiss roll data
data = swiss_roll(n)

# Create Manifold Sculpting object
manifold = ManifoldSculpting(data, n_neigh=3, d = n*2)

In [15]:
print(manifold.data.shape)
print(manifold.n_neigh)
print(manifold.d)
print(manifold.idx_neighbors.shape)
print(manifold.delta.shape)
print(manifold.theta.shape)
print(manifold.mean_delta.shape)

(1000, 3)
3
5
(1000, 3)
(1000, 3)
(1000, 3)
()
